In [174]:
#import 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [175]:
#get data and format it 
df = pd.read_csv('tweets.csv', names = ['target','id','date','flag','user','text'])
sentiment = df.pop('target').values
tweet = df.pop('text').values
train_labels, test_labels, train_tweets, test_tweets = train_test_split(sentiment,tweet)

In [176]:
#reusable formatting function
def preprocess(arr):
    #tokenize
    tokenizer = Tokenizer(num_words=15_000, oov_token='<UNK>', split = ' ')
    tokenizer.fit_on_texts(arr)

    #index
    word_index = tokenizer.word_index

    #sequence
    sequences = tokenizer.texts_to_sequences(arr)

    #find max sequence length
    maxlen = max([len(x) for x in sequences])

    #padding 
    train_padded = pad_sequences(sequences, padding='post', truncating='post', maxlen=maxlen)
    
    return train_padded

In [177]:
#preprocess 
test_labels = test_labels[:15_000]
train_labels = train_labels[:15_000]
train_tweets = train_tweets[:15_000]
test_tweets = test_tweets[:15_000]
train_tweets = preprocess(train_tweets)
test_tweets = preprocess(test_tweets)
test_labels = test_labels/2
train_labels = train_labels/2
test_labels = test_labels.reshape([-1, 1])
train_labels = train_labels.reshape([-1,1])

In [211]:
model = keras.Sequential()
model.add(keras.layers.Embedding(15_000, 20))
model.add(keras.layers.LSTM(256))
#model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(50, activation = "relu"))
#model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(3, activation = "softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [167]:
fitmodel = model.fit(train_tweets, train_labels, epochs = 5, batch_size = 50, validation_split=0.2)

Train on 12000 samples, validate on 3000 samples
Epoch 1/5
12000/12000 [==============================] - 67s 6ms/sample - loss: 0.5322 - accuracy: 0.8002 - val_loss: 0.5023 - val_accuracy: 0.7993
Epoch 2/5
12000/12000 [==============================] - 64s 5ms/sample - loss: 0.4991 - accuracy: 0.8036 - val_loss: 0.5066 - val_accuracy: 0.7993
Epoch 3/5
12000/12000 [==============================] - 64s 5ms/sample - loss: 0.4996 - accuracy: 0.8036 - val_loss: 0.5040 - val_accuracy: 0.7993
Epoch 4/5
12000/12000 [==============================] - 66s 5ms/sample - loss: 0.4979 - accuracy: 0.8036 - val_loss: 0.5082 - val_accuracy: 0.7993
Epoch 5/5
12000/12000 [==============================] - 63s 5ms/sample - loss: 0.4909 - accuracy: 0.8045 - val_loss: 0.4878 - val_accuracy: 0.8010


In [168]:
model.save('twittersent_sparsecat.h5')

In [212]:

model.load_weights('twittersent_sparsecat.h5')

In [ ]:
model.evaluate(test_tweets, test_labels)

In [ ]:
user = 'good'
while (user != 'stop'):
    print("copy paste a tweet and i'll tell you it's sentiment. Type 'stop' at any time to stop the program. ")
    user = str(input()).lower()
    user = preprocess(user)
    prediction = model.predict(user)
    sentiment0 = np.amax(prediction[0])
    sentiment1 = np.amax(prediction[1])
    sentiment2 = np.amax(prediction[2])
    if sentiment0 > sentiment1 and sentiment0 > sentiment2:
        print("this is a negative comment")
    elif sentiment1 > sentiment0 and sentiment1 > sentiment2:
        print("this is a neutral comment")
    elif sentiment2 > sentiment1 and sentiment2 > sentiment0:
        print("this is a positive comment")